In [ ]:
import missingno as msngo
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np

import lightgbm as lgb
import optuna

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

from sklearn.preprocessing import StandardScaler

# Spike Challenge: Predicción de precios de insumos básicos en Chile 

**Nicolás Caro**

En este desafío vamos a ver si somos capaces de **predecir el precio de un insumo básico, 
como la leche, a partir de variables climatológicas y macroeconómicas**. 

No siempre estos datos nos entregan toda la información que nos gustaría, como por ejemplo, señales claras 
del avance de la sequía a lo largo del país. Sin embargo, nos permiten entender otro tipo de 
efectos,  como  movimientos  en  ciertos  sectores  de  la  economía.  

En  esta  línea,  te  iremos guiando para construir un análisis y algunos modelos que nos ayuden a concluir. 

## Datos: Precipitaciones, Indicadores Económicos Banco Central 
 
* Cargar  archivo  precipitaciones.csv  con  las  precipitaciones  medias  mensuales registradas entre enero 1979 y abril 2020. (Unidad: mm). 

Usamos el argumento `parse_dates` para trabajar directamente con objetos tipo `datetime`. Más aún, el argumento `index_col` permite configurar dichas fechas como *indice* o *id* de la base. Trabajar de esta manera proporciona mejores herramientas de selección y manipulación de datos. 

Se espera que el `DataFrame` resultante se comporte como un conjunto de series de tiempo, indexadas por `date`. Donde cada columna representa una región.

In [ ]:
precipitaciones = pd.read_csv('https://raw.githubusercontent.com/SpikeLab-CL/desafio_spike_precios/c01b3f9aa6d5a5359cc6d390f57ebb53f5bb7885/precipitaciones.csv',
                              parse_dates=['date'], index_col='date')
precipitaciones.head()

Revisamos si en efecto se tienen datos entre enero 1979 y abril 2020. Para ello, se ordena el dataset según fecha y se observan el primer y último registro:

In [ ]:
precipitaciones.sort_values(by='date', inplace=True) 
precipitaciones.iloc[[0,-1]]

* Cargar archivo banco_central.csv con variables económicas.

Se carga esta base

In [ ]:
banco_central = pd.read_csv(
    'https://raw.githubusercontent.com/SpikeLab-CL/desafio_spike_precios/main/banco_central.csv')

# Ordenamos los valores para asegurar un codigo reproducible
banco_central.sort_values(by='Periodo', inplace=True)
banco_central.head()

Observamos que la columna `Periodo` presenta *aparentemente* un formato del tipo `%Y-%m-%d %H:%M:%S %Z` es decir *año - mes - dia hora:minuto:segundo Zona_horaria*. Al igual que el caso anterior, nos conviene trabajar con objetos tipo `datetime` para mejor control sobre los datos. 

Se utiliza `pd.to_datetime` entregando el formato y la columna `Periodo`:

In [ ]:
try:
    pd.to_datetime(banco_central.Periodo)
except ValueError as e:
    print(f'Format error found: \n {e}')

Es decir, tenemos fechas con formato `2020-13-01 00:00:00 UTC` donde claramente `13 > 12`, la cantidad de meses de un año. Al estudiar la estructura de los datos, vemos que en efecto, tienen la forma `%Y-%m-%d %H:%M:%S %Z`, más especificamente, `%Y-%m-01 00:00:00 UTC`donde `%Y` y `%m` representan año y mes. En este contexto, se busca identificar aquellos indicies para los cuales hay meses mayores a 12. Para ello se ejecuta:

In [ ]:
bad_indexes = []
for i in range(banco_central.shape[0]):
    try: 
        pd.to_datetime(banco_central.Periodo.iloc[i]) 
    except: 
        idx = banco_central.iloc[i].name 
        print(f'Error at row {i}, with index {idx}')
        bad_indexes.append(i) 

Es decir, solamente la observación `613` con identificador `89` presenta dicho problema. Esto se tratará posteriormente. 

## Análisis de datos: Creación de variables

* Realiza un análisis exploratorio de la base de datos, ¿Qué puedes decir de los datos, sus distribuciones, valores faltantes, otros? ¿Hay algo que te llame la atención? 

Comenzamos con la base `precipitaciones`

In [ ]:
precipitaciones.info()

La base tiene 496 registros, estos están indexados por fecha desde  `1979-01-01` hasta `2020-04-01`. Con lo que se confirma la primera revisión que se hizo de los datos. Hay información sobre 8 columnas, cada una de las cuales representa una región de Chile. A priori no se aprecian valores faltantes, el tipo de dato para cada región es del tipo `float64`.

Se estudia la estructura preliminar de los datos:

In [ ]:
precipitaciones.describe().T 

La media de precipitaciones coincide con la ubicación geográfica de cada región. De la misma manera, los máximos y minimos tienen relación con el panorama que uno esperaría a priori para cada región. 

En cuanto a las propiedades estadísticas del conjunto de datos, se puede observar que el grado de dispersión en la caida de lluvias es alto, pues la desviación estandar es en la mayoria de los casos mayor o bastante similar a la media para cada región.

Se observa la distribución de los posibles valores faltantes en el conjunto datos:

In [ ]:
fig, ax = plt.subplots()
msngo.matrix(precipitaciones, fontsize=15, figsize=[ 
             8, 6], ax=ax, sparkline=False);

ax.set_title('Missing Values Rain Dataset', fontsize = 20); 

En efecto, la visualización anterior confirma la ausencia de valores faltantes. En cuanto a los valores duplicados:

In [ ]:
precipitaciones.index.duplicated().sum()

Se ve que no existen entradas con valores duplicados.

Hasta ahora, tanto los estadísticos de tendencia central como las visualizaciones e inspecciones iniciales, indican que el conjunto de datos asociado a las precipitaciones se encuentra bastante *limpio*. 

Pasamos a estudiar las distribuciones univariadas para cada región

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=[12, 12])

# Se elimina el ultimo Subplot
ax[-1, -1:][0].remove()

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Rain Dataset: Univariate Distributions ',
             fontsize=20,
             x=0.5,
             y=1.02)

for axis, col in zip(ax.flatten(), precipitaciones.columns):
    sns.histplot(precipitaciones[col], ax=axis, bins=50, stat='density',
                 fill=False)
    axis.set_xlabel(col, fontsize=15)
    axis.set_ylabel('', fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.22, .22)
plt.subplots_adjust(wspace=w, hspace=h)

los datos de la base `precipitaciones` corresponden a medias *mensuales* en *mm* para cada región durante `496` meses. En este contexto, la visualizanción anterior nos muestra una fotografia **estática** y **global** de lo que ocurrió durante este periodo (extenso) de tiempo y no nos permite observar la **dinamica** anual o sobre algún periodo significativo (ej: sabemos que en chile, durante el verano llueve menos que en invierno). No obstante, se puede apreciar que las regiones `Coquimbo`, `Valparaiso`,`Metropolitana_de_Santiago`, `Libertador_Gral__Bernardo_O_Higgins` y `Maule`son en general más secas que `Biobio`, `La_Araucania` y `Los_Rios`. Siendo estás últimas dos regiones las más lluviosas del conjunto de datos. 

Podemos visualizar la distribución de lluvias desde la perspectiva mensual. Para ello, reagrupamos los datos según mes y vemos la distribución de lluvias por región:

In [ ]:
months = precipitaciones.index.month_name()
query_df = precipitaciones.groupby(by=months)

fig, ax = plt.subplots(nrows=4, ncols=3, figsize=[
                       12, 12], sharex=True, sharey=True)

for axis, mon in zip(ax.flatten(), months):
    
    axis.set_title(mon, fontsize = 15)
    sns.boxplot(data=query_df.get_group(mon), orient='h',
                palette='pastel', ax=axis) 
    axis.tick_params(labelsize = 13)
    
fig.suptitle('Rain Dataset: Distributions per Month',
             fontsize=20,
             x=0.4,
             y=1.02);

fig.tight_layout()

En este caso, se observa la progresión y regresión estadística de las lluvias en cada región para un año *estadístico*. Claramente se aprecia como las lluvias son menores en los meses de verano para aumentar en otoño e invierno sin importar la región. 

Por último, observamos algunas medidas generales para cada mes en todo el terrotorio.

In [ ]:
grouped_df = precipitaciones.groupby(by = precipitaciones.index.month)

agg_df = grouped_df.mean().T 
agg_df.columns = months.unique()

monthly_metrics = agg_df.describe()
monthly_metrics.drop('count', inplace=True)
monthly_metrics

donde se aprecia una alta variación a lo largo del territorio nacional para cada mes, además de una mayor concentración de las lluvas durante los meses de mayo, junio, julio y agosto. 

De manera gráfica esto se expresa según:

In [ ]:
fig, ax = plt.subplots(figsize=[12, 10])

sns.boxplot(data=agg_df, ax=ax, orient='h', palette='pastel')
ax.set_title('Rain Distribution Over a Statistical Year', fontsize=20)
ax.tick_params(labelsize = 18)

En cuanto a la base de precipitaciones:

1. ¿Qué puedes decir de los datos, sus distribuciones, valores faltantes, otros? 

Se puede inferir que independiente de la región, cada año presenta una gran dispersión en cuanto a los *mm* de lluvia caidos. Las distribuciones observadas concuerdan con el comportamiento esperado para cada región y año estadístico. Desde el punto de vista técnico, el conjunto de datos no presenta problemas estructurales del tipo *valores perdidos*, *valores duplicados* ni *incosistencias* en *tipos de datos*, ni *distribuciones*.


2. ¿Hay algo que te llame la atención? 

Me llamó la atención que la gran parte de los *mm* de lluvia caidos durante el año se concentran a nivel pais solo en 4 meses (mayo - agosto), además de la alta densidad de valores cercanos a 0 *mm* desde Coquimbo hasta el Bio-Bio.

Continuamos con la base `banco_central`. 

Anteriormente observamos que había una observación cuyo mes estaba identificado como `13`. Estudiaremos la estructura del dataset para ver si este dato erroneo es remediable. Para ello, estudiamos la cantidad de entradas por año. 

Primero aislamos la observación con problemas

In [ ]:
iso_obs = banco_central.loc[89].copy()
banco_central.drop(index=89, inplace=True)

Luego configuramos `Perdiodo` como indice de la base y transformamos a formato `datetime`.

In [ ]:
banco_central.set_index(keys='Periodo',inplace=True)
banco_central.index = pd.to_datetime(banco_central.index)
banco_central.index = banco_central.index.tz_localize(None)

Finalmente agrupamos por año y vemos la cantidad de entradas por cada año. Para esto se selecciona una columna y se extrae la cantidad de observaciones (con o sin datos) de cada año para dicha variable. 

In [ ]:
query_df = banco_central.groupby(banco_central.index.year)
agg_df = query_df['Precio_de_la_onza_troy_de_oro_dolaresoz'].describe()['count']
agg_df

Podemos observar que en general se tienen 12 entradas por año, una asocidad a cada mes. La excepción son los años `2018`, `2019` con 13 entradas y `2020` con 11. Podemos asumir que la entrada con mes `13` asilada previamente, corresponde al mes 12 del año 2020 con alta probabilidad (dada la estructura del dataset). Para confirmar esto, podemos agrupar por mes y ver si el mes 12 tiene menos entradas asociadas por año:

In [ ]:
def get_month_counts(df=banco_central):
    """Returns a pd.Series object summarizing entries per month.

        Parameters:

        df (pd.DataFrame): dataframe to work with, It's intended to have the same 
            structure as the default dataset.

        Returns:
        pd.Series: df summary.

    """
    # We count the number of entries
    def entry_counter(x): return x.shape[0]

    query_df = df.groupby(df.index.month)
    agg_df = query_df.aggregate(entry_counter)

    agg_df.index = ['January', 'February', 'March', 'April', 'May', 'June',
                    'July', 'August', 'September', 'October', 'November',
                    'December']

    df_res = agg_df.iloc[:, 0].copy()
    df_res.name = 'Entries'

    return pd.DataFrame(df_res)

Vemos que en efecto el mes 12 (December) posee 50 entradas, a diferencia de las 51 presentes para cada mes, excepto el mes 8 (August). La presencia de 53 entradas en el mes 8 se puede deber a observaciones duplicadas. 

Quitamos valores duplicados:

In [ ]:
banco_central.drop_duplicates(inplace=True)

In [ ]:
get_month_counts(df = banco_central)

Con lo anterior observamos que en efecto el mes de agosto pasa a tener los 51 registros esperados. Finalmente agregamos la observación aislada como mes 12:

In [ ]:
iso_obs.Periodo = '2020-12-01'
iso_obs.name = pd.to_datetime(iso_obs.Periodo) 
iso_obs.drop('Periodo', inplace=True) 

banco_central = banco_central.append(iso_obs)

In [ ]:
get_month_counts(df = banco_central)

Con lo que confirmamos que se ha agregado el dato de manera correcta.

A continuación se estudia un poco más a fondo la estructura del dataset. Comenzamos por chequear los tipos de dato:

In [ ]:
# Para que coincida con el indice de precipitaciones
banco_central.index.name = 'date'
banco_central.sort_index(inplace=True)
banco_central.info()

Preliminarmente, se puede apreciar que existen 84 variables, y 612 registros. Se posee información entre 1970-01-01 al 2020-12-01. 

En general las variables son reconocidas como tipo `object`, es decir, no se puede reconocer un tipo de dato especifico para dichas variables. Además se observa que la cantidad de valores no nulos por columna es relativamente bajo. 

Verificamos la estructura de valores nulos:

In [ ]:
msngo.matrix(df=banco_central, fontsize=25)

Dado que el conjunto de datos está ordenado, podemos inferir que para los datos más cercanos al 2020 son aquellos con mayor información recolectada. Se puede ver además que existe una gran cantidad de valores faltantes (en blanco se muestran los valores faltantes, en negro los valores presentes).

Estudiamos más en profundidad las variables involucradas en este dataset.



In [ ]:
print(banco_central.columns.values)

Como vimos anteriormente, en su mayoria estas variables son reconocidas como objetos tipo `object`. Buscamos transformar dichas columnas a un tipo de datos que se condiga con la información de la base. 

Una inspección simple del dataset muestra que en su mayoria, los valores en las columnas reconocidas como `object` poseen el formato `xxx.yyy.zzz`. En efecto:

In [ ]:
banco_central.select_dtypes('object').dropna().head(10)

Podemos eliminar los puntos para obtener datos del tipo numerico.

In [ ]:
obj_cols = banco_central.select_dtypes('object').columns

banco_central[obj_cols] = banco_central[obj_cols].apply(
    lambda x: x.str.replace(".", "", regex=False))

err_cols = []
for col in banco_central.columns:
    try:
        banco_central[col] = banco_central[col].astype(float) 
    except ValueError as e: 
        print(f'Columna {col} presenta el error: \n {e}')      
        err_cols.append(col)

Lo anterior nos indica que restan columnas para las cuales aparece el valor `'a'`. Este se reemplaza por `None`, finalmente se terminan de convertir dichas columnas al tipo `float`.

In [ ]:
banco_central[err_cols] = banco_central[err_cols].replace({"a":None}) 
banco_central[err_cols] = banco_central[err_cols].astype(float) 

Como resultado, ya no existen columnas reconocidas como tipo `object`:

In [ ]:
len(banco_central.select_dtypes('object').columns)

Los nombres de las variables nos indican que las columnas representan grupos de indices como lo son el IMACEC, PIB e indicadores estadisticos del INE entre otros.

Usando la información anterior, estudiaremos los datos faltantes, agrupando según lo sugieren las columnas involucradas. Para ello, generamos la serie `missing_series`, que almacena el porcentaje de valores faltantes para cada columna:

In [ ]:
total = banco_central.shape[0]
def f_perc(x): return round((x/total)*100, 1)

missing_series = f_perc(banco_central.isnull().sum())
missing_series.name = 'missing_percentage'

Es posible consultar de manera sencilla el porcentaje de valores faltantes según agrupación de interés. 

In [ ]:
# Busca una palabra de interés en las columnas de la base del banco central
def miss_string_finder(string): return [
    i for i in missing_series.index.values if string in i]

In [ ]:
#Imacec
missing_series[miss_string_finder('Imacec')]

In [ ]:
#PIB
missing_series[miss_string_finder('PIB')]

In [ ]:
#INE
missing_series[miss_string_finder('INE')]

Para estos ejemplos se observa un porcentaje de perdida de información superior al 50%. Se procede a estudiar cuantas variables poseen una cantidad de información mayor a ese porcentaje:

In [ ]:
missing_series[missing_series < 50]

Es decir, solo 10 columnas poseen más de la mitad de los registros, teniendose en `precio_de_la_gasolina_en_eeuu_dolaresm3`,`precio_de_la_onza_troy_de_oro_dolaresoz`,`precio_de_la_onza_troy_de_plata_dolaresoz` y `precio_del_cobre_refinado_bml_dolareslibra` información casi completa.

Eliminamos aquellas columnas con mas del 90% de datos faltantes:

In [ ]:
to_drop = missing_series[missing_series > 90].index
banco_central.drop(columns=to_drop.values, inplace=True)

Para concluir el análisis exploratorio y limpieza de la base del banco central, estudiamos las distribuciones de los datos según grupo de indices:

In [ ]:
def string_finder(string): return [
    i for i in banco_central.columns.values if string in i]


def group_plotter(string):
    '''Creates a boxplot containing a string-data group'''
    
    fig, ax = plt.subplots(figsize=[12, 10])

    idx = string_finder(string)
    df = banco_central[idx]

    sns.boxplot(data=df, ax=ax, orient='h', palette='pastel')

    ax.set_title(string + ' Index Group', fontsize=20)
    ax.tick_params(labelsize=18)

In [ ]:
# Imacec
group_plotter('Imacec')

No se aprecian distribuciones muy distintas para estos indices, a pesar de la escala de los datos, se puede decir que se encuentran bien distribuidos. La diferencia en las medias de este grupo puede aportar varianza a la hora de predecir una variable de respuesta.

In [ ]:
# PIB
group_plotter('PIB')

Se observa escalas diferentes a pesar de pertenecer al mismo grupo de indices, es posible que incluir este grupo de variables introduzca mucho ruido a un posible modelo predictivo.

In [ ]:
# Precio bienes 
group_plotter('Precio')

Se observa un cambio de escala significativo entre distintas categorias, por tal motivo, tiene más sentido estudiar según cantidad fisica asociada:

In [ ]:
# Precio bienes -> onzas, m3, barril
group_plotter('oz')
group_plotter('m3')
group_plotter('barril')


Entre el oro y la plata se aprecia una gran diferencia en distribución, siendo la plata significativamente menor. Por otra parte, el metro cubico de kerosene y parafina se comportan de manera similar en cuanto a su magnitud. Estaditicamente, la gasolina posee una media mucho más baja y concentrada. Finalente los barriles de petroleo presentan diferencias en sus medias y pocos valores fuera de rango, es probable que estas variables aporten positivamente al modelo.

In [ ]:
# Indices INE
group_plotter('INE')

Grupo bastante heterogeneo, esto se puede relacionar a la naturaleza de cada indicador.

In [ ]:
# Indices 
group_plotter('Indice')

En general se observa un buen comportamiento y similitud en escala, excepto para el indicie de producción industruial de electricidad, gas y agua. 

1. ¿Qué puedes decir de los datos, sus distribuciones, valores faltantes, otros? 

A diferencia de la base de precipitaciones, la base del banco central presenta datos heterogeneos, esto se ve al observar los indicadores estadisticos de dispersión. En cuanto a los valores faltantes, esta base consta en su mayoria de elementos de dicho tipo. Más aún, la escala y los procedimiento ultilizados para limpiar esta base, hacen creer que pudo haber ocurrido un error al importar o recolectar los datos.

2. ¿Hay algo que te llame la atención? 

Me llamó la atención como la escala de los valores fue transversal al transformar a tipo de dato numerico, esto debido a la naturaleza heterogenea de los datos. En otras palabras, hubiera esperado más cambios de escala. Nuevamente, esto me hace sospechar del proceso de obtención de los datos.

## Visualización 

* Crea una función que permita graficar series históricas de precipitaciones para un 
rango de fechas determinado. Para esto la función debe recibir como argumentos el 
nombre de una región, fecha de inicio y fecha de término (asegúrate de verificar en 
tu función que tanto el nombre de la región como las fechas ingresadas existan en el 
dataset).  

In [ ]:
def rain_plot(region_name, start_date, end_date):
    '''Creates a lineplot to visualize rain data.

    Parameters:

        region_name (str): Region to visualize, must be one of these:
            'Coquimbo' 'Valparaiso' 
            'Metropolitana_de_Santiago'
            'Libertador_Gral__Bernardo_O_Higgins' 
            'Maule' 
            'Biobio' 
            'La_Araucania'
            'Los_Rios'

        start_date (str or datetime): start date, must be >= 1979-01-01.
        end_date (str or datetime): end date, must be <= 2020-04-01.

    '''
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Date Check

    s = precipitaciones.index.min()
    e = precipitaciones.index.max()
    date_range = pd.date_range(start=s, end=e)

    t_msg = f'Invalid date, It must be between {s} and {e}'

    assert start_date <= end_date, 'start_date must be <= end_date'
    assert start_date in date_range, t_msg
    assert end_date in date_range, t_msg

    # Region Check

    regs = precipitaciones.columns.unique()
    r_msg = f'region_name must be one of these: {regs.values}'

    assert region_name in precipitaciones.columns, r_msg

    # Data gathering
    data = precipitaciones[region_name].loc[start_date:end_date]

    # Plot
    fig, ax = plt.subplots(figsize = [15,8])
    sns.lineplot(data=data, ax=ax)
    
    ax.set_title(f'Rains (mm) in {region_name}', fontsize=20)
    ax.tick_params(labelsize=18)
    ax.set_xlabel('date', fontsize=18)
    ax.set_ylabel('Rain (mm)', fontsize=18)

* Usa esta función para graficar las precipitaciones para la Región Libertador General 
Bernardo O'Higgins y para la Región Metropolitana entre las fechas 2000-01-01 y 2020-01-01. 

In [ ]:
rain_plot('Libertador_Gral__Bernardo_O_Higgins', ' 2000-01-01', '2020-01-01')

In [ ]:
rain_plot('Metropolitana_de_Santiago', ' 2000-01-01', '2020-01-01')

* ¿ Qué observas con respecto a estacionalidades y tendencias? 

Se ve un comportamiento bastante similar en ambas regiones, se hace evidente la tendencia a la baja en ambas series para los ultimos 20 años, lo cual es un posible indicador de sequia.

En cuanto a la estacionalidad, esta es clara para ambas regiones, pues se observa una componente periodica con picks y valles recurrentes aunque con menor intensidad a medida que pasan los años. 

* Crea  una  función  que,  para  una  región,  grafique  múltiples  series  de  tiempo mensuales de precipitaciones, donde cada serie de tiempo corresponda a un año. La función debe recibir como argumento una lista con los años que queremos graficar (2000,  2005,..)  y  el  nombre  de  la  región.  El  eje  X  debe  indicar  los  meses  (enero, febrero, etc...).  

In [ ]:
def monthly_rain_plot(region_name, year_list):
    '''Creates a lineplot to visualize monthly rain data.

        Parameters:

        region_name (str): Region to visualize, must be one of these:
            'Coquimbo' 'Valparaiso'
            'Metropolitana_de_Santiago'
            'Libertador_Gral__Bernardo_O_Higgins'
            'Maule'
            'Biobio'
            'La_Araucania'
            'Los_Rios'

        year_list (list): List of (int or string) years start date, must be 
        >= 1979-01-01 and end_date must be <= 2020-04-01.

    '''

    # Date Check
    s = precipitaciones.index.min()
    e = precipitaciones.index.max()
    date_range = pd.date_range(start=s, end=e)

    t_msg = f'Invalid year, It must be between {s} and {e}'

    for y in year_list:
        assert str(y) in date_range, t_msg

    # Region Check
    regs = precipitaciones.columns.unique()
    r_msg = f'region_name must be one of these: {regs.values}'

    assert region_name in precipitaciones.columns, r_msg

    # Data Gathering
    get_years = [y for y in precipitaciones.index if y.year in year_list]
    data = precipitaciones.loc[get_years, region_name]

    year_series = pd.Series(data.index.year, name='year', index=data.index)
    data = pd.concat([data, year_series], axis=1)

    # Plot
    fig, ax = plt.subplots(figsize=[15, 8])
    sns.lineplot(x=data.index.month_name(),
                 y=data[region_name], hue=data.year, ax=ax, palette='rocket_r')

    ax.set_title(f'Monthly Rains (mm) in {region_name}', fontsize=20)
    ax.tick_params(labelsize=13)
    ax.set_xlabel('Month', fontsize=18)
    ax.set_ylabel('Rain (mm)', fontsize=18)

* Usa esta función para graficar las precipitaciones para la Región del Maule durante 
los años 1982, 1992, 2002, 2012 y 2019. 

In [ ]:
monthly_rain_plot("Maule", [1982, 1992, 2002, 2012, 2019])

* ¿Qué puedes concluir de estos gráficos? 

Se observa una clara disminución en la cantidad de precipitaciones a los largo de las decadas. Aunque en el 2002 hubo un peak en un mes normalmente seco (agosto), la norma es que los meses mas lluviosos (mayo, junion) decaen gradualmente en cuanto a la cantidad de *mm*.


* Crea una función que permita visualizar dos series históricas de PIB para un rango de 
fechas determinado. Para esto la función debe recibir como input el nombre de cada serie, fecha de inicio y fecha de término. 

In [ ]:
def pib_2_series_plot(series_1_name, series_2_name, start_date, end_date):
    '''Creates a lineplot to visualize two PIB-timeseries.

        Parameters:

        series_1 (str): First series tio visualize.
        series_2 (str): Second series tio visualize.

        start_date (str): Start date to plot.
        end_date (str): End date to plot. 

        

        Allowed series values are: 

        'PIB_Agropecuario_silvicola',
        'PIB_Pesca',
        'PIB_Mineria',
        'PIB_Mineria_del_cobre',
        'PIB_Otras_actividades_mineras',
        'PIB_Industria_Manufacturera',
        'PIB_Alimentos',
        'PIB_Bebidas_y_tabaco',
        'PIB_Textil',
        'PIB_Maderas_y_muebles',
        'PIB_Celulosa',
        'PIB_Refinacion_de_petroleo',
        'PIB_Quimica',
        'PIB_Minerales_no_metalicos_y_metalica_basica',
        'PIB_Productos_metalicos',
        'PIB_Electricidad',
        'PIB_Construccion',
        'PIB_Comercio',
        'PIB_Restaurantes_y_hoteles',
        'PIB_Transporte',
        'PIB_Comunicaciones',
        'PIB_Servicios_financieros',
        'PIB_Servicios_empresariales',
        'PIB_Servicios_de_vivienda',
        'PIB_Servicios_personales',
        'PIB_Administracion_publica',
        'PIB_a_costo_de_factores',
        'PIB'
        
        Start and End dates must be >= 1979-01-01 and <= 2020-04-01.            
    '''
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Date Check

    s = banco_central.index.min()
    e = banco_central.index.max()
    date_range = pd.date_range(start=s, end=e)

    t_msg = f'Invalid date, It must be between {s} and {e}'

    assert start_date <= end_date, 'start_date must be <= end_date'
    assert start_date in date_range, t_msg
    assert end_date in date_range, t_msg

    # PIB Series name check
    allowed_names = string_finder('PIB')

    def msg(n): return f'Series {n} name must be one of these {allowed_names}'

    assert series_1_name in allowed_names, msg(1)
    assert series_2_name in allowed_names, msg(2)
    
    # Data Gathering
    ser_1 = banco_central[series_1_name][start_date:end_date]
    ser_2 = banco_central[series_2_name][start_date:end_date]

    data = pd.concat([ser_1,ser_2], axis=1)
    data.dropna(inplace=True)
    
    # Plot
    fig, ax = plt.subplots(figsize=[15, 8])
    sns.lineplot(data=data, ax=ax)

    ax.set_title(f'PIB indexes', fontsize=20)
    ax.tick_params(labelsize=15)
    ax.set_xlabel('Date', fontsize=18)
    ax.set_ylabel('Index Value', fontsize=18)

* Grafica las series de tiempo del PIB agropecuario y silvícola y la del PIB de Servicios 
financieros desde el 2013-01-01 hasta la fecha más reciente en que haya datos. 

In [ ]:
#Obs, basta con configruar end_date = 2020-12-01, la funcion elimina los na
pib_2_series_plot('PIB_Servicios_financieros',
                  'PIB_Agropecuario_silvicola', '2013-01-01', '2020-12-01')

* ¿Qué puedes decir de cada serie en particular? 

Se puede observar cierta componente periodica en ambos indeces, en cuanto al PIB Agropecuario, se aprecia cierta estacionalidad similar a la de lo *mm* caidos de manera anual a nivel pais. es posible que exista una alta correlación entre ambas variables. Por otra parte, el PIB de servicios financieros tiende levemente al alza superando los peaks del PIB agropecuario a medida a partir del año 2019. Esto ultimo indica un comportamiento distinto al apreciado entre el 2013 y 2018. ES posible que en 2021 y 2022 esta diferencia se acentúe. 

* ¿Hay alguna relación entre estas dos series?

Ambas disminuyen en los mismos periodos temporales, si el PIB de servicios financieros es más estable durante el año, su magnitud en similar al peak alcanzado por el PIB agropecuario.

## Tratamiento y creación de variables 

* ¿Cómo podríamos evaluar la correlación entre las distintas series de tiempo y cómo se  tienen  que  correlacionar  para  entrenar  un  modelo?  ¿Mucha  correlación,  no correlacionadas, da igual? 

Se puede medir la correlación entre series de tiempo estudiando la correlación (valga la redundancia) entre los valores de estas, para distintas frecuencias. Es decir, podemos agregar los datos, por ejemplo, según mese, semestre, año, decada, etc... dando lugar a nuevas series de tiempo. Estas series representan un conjunto de valores con diferentes componentes de información acerca de las series de tiempo originales. Podemos así, medir la auto-correlación de los valores en cada intervalos de agregación, es decir, la similitud lineal de una serie con sigo misma para distintos intervalos de tiempo. Como tambien podemos medir correlaciones cruzadas, es decir, la similitud lineal entre ventanas temporales de una serie con las ventanas temporales de otra serie. 

En general, la correlación es un medición estadistica de *similitud lineal*, podemos tambien utilizar otras medidas de similitud, como por ejemplo la *información mutua*, el coeficiente de correlación de Kendall, funciones kernel, entre otras. 

Cuando se observan correlaciones en una base de datos, se espera que las variables explicativas se correlacionen o posean cierto grado de similitud (bajo algún criterio) con la variable a predecir. Para eliminar ruido de un modelo y hacerlo más simple, se busca también que la corelación (o similitud arbitraria) entre variables explicativas sea baja, un ejemplo cotidiano es el problema de *colinealidad* entre variables explicativas cuando se usa un modelo lineal. 

Actualmente se buscan modelos capaces de capturar *causalidad* en el modelo de datos, lamentablemente, las correlaciones representan medidas de similitud entre variables más bien geométricas. En este conexto, podriamos tener variabls altamente correlacionadas pero sin relación lógica alguna. Por tal motivo, la existencia de correlaciones puede ser vista como una característica positiva en un modelo de datos, mas no obligatoria. 

* Para el entrenamiento del modelo, queremos predecir el precio de la leche para el productor en Chile. Para eso, descarga el archivo precio_leche.csv y haz un merge con las bases de datos de precipitaciones y datos del Banco Central. 

In [ ]:
leche = pd.read_csv('https://raw.githubusercontent.com/SpikeLab-CL/desafio_spike_precios/main/precio_leche.csv')
leche.head()

In [ ]:
leche.info()

In [ ]:
msngo.matrix(leche)

La base no posee valores faltanes ni duplicados.

In [ ]:
resp = not leche.drop_duplicates().shape == leche.shape
print(f'Has duplicated values? {resp}')

 Procedemos a indexar por fecha.

In [ ]:
mapping = {mes:num for num,mes in enumerate(leche['Mes'].unique(), start=1)}
leche['Mes'].replace(mapping, inplace=True)
dates = leche['Anio'].astype(str) + '-' + leche['Mes'].astype(str)

leche.index = pd.to_datetime(dates)
leche.index.name = 'date'

leche.drop(columns=['Anio','Mes'], inplace= True)

In [ ]:
leche.head()

Agregamos las bases correspondientes

In [ ]:
dataset = pd.merge(precipitaciones, banco_central,
                  left_index=True, right_index=True, how='left')
dataset = pd.merge(leche, dataset, left_index=True, right_index=True, how='left')
dataset.head()

In [ ]:
dataset.info()

* Crea las variables: 
    * A partir de la variable fecha, crea nuevas variables para el año, mes, trimestre. 

In [ ]:
# Año
dataset['year'] = dataset.index.year.astype(str) 

#Mes
dataset['month'] = dataset.index.month_name().astype(str) 

#Trimestre
mapping = {'January': 1, 'February': 1, 'March': 1, 'April': 2, 'May': 2, 'June': 2, 'July': 3,
           'August': 3, 'September': 3, 'October': 4, 'November': 4, 'December': 4}
dataset['tri'] = dataset['month'].replace(mapping)

 * Lags  y  estadísticas  acumuladas  (por  ejemplo:  promedio,  varianza)  de  las variables que consideres relevantes.
 
En primer lugar, podemos agregar la columna `y` que consta del precio de la leche mensual adelantada en un periodo. De esta forma, damos estructura al dataset, de manera tal que los datos actuales nos permitan predecir el precio de la leche del mes siguiente.

In [ ]:
dataset['y'] = dataset['Precio_leche'].shift(1)
dataset.drop(labels='Precio_leche', axis = 1, inplace=True)
dataset.sort_index(inplace=True)

dataset = dataset[~dataset['y'].isna()] 
y = dataset['y']

Dado el comportamiento periodico anual observado en las bases de precipitaciones y banco_central, tiene sentido calcular estadisticos anuales en este dataset. en este contexto, podemos calcular las medias simples, minimos y maximos anuales en precipitaciones.

In [ ]:
summary_cols = {'year', 'month', 'tri'}
to_aggregate = dataset.columns.difference(summary_cols)

for col in to_aggregate:
    data = pd.concat([dataset[col].rolling(window=12).mean(),
                         dataset[col].rolling(window=12).max(),
                         dataset[col].rolling(window=12).min()], axis=1) 
    data.columns  = [col + '_year_mean', col + '_year_max', col + '_year_min']
    dataset = pd.merge(dataset, data, left_index=True, right_index=True, how = 'left') 

Finalmente estudiamos las correlaciones en el conjunto de datos con la variable objetivo:



In [ ]:
dataset.corr()[['y']].drop('y').sort_values(by=['y'], ascending=False)

Se observa una alta correlación con los valores del precio de la leche del año pasado, ya sean en su minimo, maximo o media. De la misma manera el imacec se correlaciona de buena manera. 

Pasaremos a eliminar las variables con correlación inferior en valor absoluto a 0.3. Para ello:

In [ ]:
cors = dataset.corr()[['y']].drop('y')
keep_cols = cors[np.abs(cors) >= 0.3].index 
dataset = dataset[keep_cols]

Eliminamos aquellas observaciones con na

In [ ]:
dataset = dataset.dropna(axis=1,how='all') 
dataset = dataset.dropna(axis=0,how='all')  

dataset

Estudiamos los valores faltantes del nuevo conjunto de datos

In [ ]:
msngo.matrix(dataset)

Observamos que hay una proción de los datos con baja cantidad de valores faltantes, dado que el conjunto de datos esta ordenado, podemos ver que esto ocurre en los últimos años. En efecto:

In [ ]:
nulls_by_year = dataset.isnull().sum(axis=1)
nulls_by_year = nulls_by_year.groupby(nulls_by_year.index.year).sum()
nulls_by_year.nsmallest(10)

Esto nos dice que entre los años 2015 al 2019 no hay valores falntantes. Procedemos a generar un conjunto de entrenamiento para estos años.

In [ ]:
X = dataset.loc['2015-01':'2019-12']

## Modelo 

* Entrena un modelo que permita predecir el precio de la leche el próximo mes, en función de los datos entregados. 

Vamos a entrar un modelo basado en `LightGBM`, este tipo de modelos generará predicciones puntuales para el precio de la leche dentro de un mes. 

En general [LightGBM](https://github.com/Microsoft/LightGBM) es en metodo basado en *gradient booting* basado en arboles de descición, es decir, consiste en un modelo de ensamblaje (o *ensemble method*) en el cual más modelos son agregados de manera que en cada iteración, se entrena un nuevo modelo basado en el error del ensamblaje completo. 

El siguiente [tutorial](https://www.frontiersin.org/articles/10.3389/fnbot.2013.00021/full) detalla más al respecto. La ventaja de LightGBM se basa en que posee una alta velocidad de entrnamiento, poco uso de memoria, posee soporte para GPU y puede trabajar con datos de gran escala. 

Para implementar dicho modelo utilizaremos [optuna](https://optuna.org/) el cual es un software de optimización diseñado para la busqueda de hiperparámetros de manera dinamica. 

Generamos un objeto `study` donde incluimos un esquema de validación curzada. Dado que los datos están agregados y se incluye un lag `y` para predecir, podemos hacer uso de la naturaleza tabular del conjunto de datos con lo que no hay problemas (realtivos a series de tiempo) en cuanto a la partición del conjunto de datos.

In [ ]:
def objective(trial):
    '''Objective function for the model optimization.'''
    
    #cross Val scheme
    train_x, valid_x, train_y, valid_y = train_test_split(
        dataset, y, test_size=0.25)
    
    #feature engeneering
    scaler = StandardScaler()
    scaler.fit(train_x)
    
    train_x = scaler.transform(train_x)
    valid_x = scaler.transform(valid_x)
    
    dtrain = lgb.Dataset(train_x, label=train_y)

    # hiper params to be tunned
    param = {
        "metric": "rmsle",
        'n_estimators': trial.suggest_int('n_estimators',10,100),
        "verbosity": -1,
        "boosting_type": "gbdt",
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth',3,10),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "seed": 1,
        "lambda_l2": trial.suggest_float("lambda_l2", 0., 10),
        'deterministic': True
    }

    model = lgb.LGBMRegressor(**param)

    gbm = model.fit(train_x, train_y)

    preds = gbm.predict(valid_x)
    err = mean_squared_log_error(valid_y, preds)
    
    return err

Entrenamos el modelo de manera paralela:

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, n_jobs=-1, show_progress_bar=True)

* ¿Qué datos adicionales te gustaría tener?¿Qué datos son necesarios para que este 
modelo funcione/mejore las métricas? 

En general seria util poseer datos relacionados a indices agropecuarios y climaticos, para mejorar las métricas de predicción es escencial que los datos sean importados de manera correcta y presenten buena correlación / causalidad con la veriable explicativa. 

* ¿Cómo evalúas el resultado del modelo?¿Qué métricas tiene sentido mirar? 

El resultado del modelo se evalua utilizando la raiz del error logaritmico medio. En este tipo de problemas tiene sentido ver métricas del tipo l2 (RMSE) o l1 (MAE) pues cuantifican (dis)concordancia entre series o suseciones. 

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Evaluamos el rendimiento en un nuevo conjunto de test.

In [ ]:
mod = lgb.LGBMRegressor(**trial.params)

train_x, test_x, train_y, test_y = train_test_split(
    dataset, y, test_size=0.25)

scaler = StandardScaler()
scaler.fit(train_x)


mod.fit(scaler.transform(train_x), train_y) 

y_pred = mod.predict(scaler.transform(test_x))

In [ ]:
fig, ax = plt.subplots(figsize=(13,10))

sns.lineplot(data=test_y,ax=ax, label = 'test') 
sns.lineplot(x=test_y.index, y= y_pred, ax=ax, label = 'pred')

ax.set_title('y Test vs y Pred for Milk Price',fontsize = 20)
ax.tick_params(labelsize=15)
ax.legend(fontsize=13);

Visualizamos las caracteristicas más importantes

In [ ]:
fig, ax = plt.subplots(figsize=(13,10))
lgb.plot_importance(mod, max_num_features=20, ax = ax,) 

ax.set_title('Feature importance', fontsize = 20)
ax.set_xlabel('Importance',fontsize = 20)
ax.set_ylabel('Features',fontsize = 20)
ax.tick_params(labelsize=15)

Este tipo de modelos puede servir para cuantificar el efecto climatico en la economia y como esto afecta la vida de las personas de manera directa. En este sentido, se pueden crear modelos similares para otros insumos de la canasta básica y mostrarlos al público y entidades publicas para generar concsciencia ambiental.